# AI Red Teaming for Azure AI Agents

This notebook demonstrates how to perform automated red teaming security testing against an Azure AI Agent using the Azure AI Evaluation SDK. We'll test the agent's responses against various attack strategies to identify potential vulnerabilities.

**Copyright (c) Microsoft Corporation. Licensed under the MIT License.**

## 1. Import Required Dependencies

We start by importing all necessary libraries for authentication, AI services, and red teaming capabilities. We'll suppress syntax warnings from third-party dependencies to keep our output clean.

In [1]:
from typing import Optional, Dict, Any
import os
import time
import warnings
from pathlib import Path
from dotenv import load_dotenv

# Suppress SyntaxWarning from third-party dependencies
warnings.filterwarnings("ignore", category=SyntaxWarning)

# Azure imports for authentication and AI services
from azure.identity import DefaultAzureCredential
from azure.ai.evaluation.red_team import RedTeam, RiskCategory, AttackStrategy
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import ListSortOrder

## 2. Load Environment Configuration

We need to load configuration from environment variables including the Azure AI Project endpoint and the name of the agent we want to test. This function locates the `.env` file and validates that all required settings are present.

In [2]:
def load_environment_configuration() -> Dict[str, Optional[str]]:
    """
    Load and validate environment variables required for the red teaming operation.
    
    Returns:
        Dict containing:
            - project_endpoint: Azure AI Project endpoint URL
            - agent_name: Name of the agent to test
    """
    # Load environment variables from .env file
    # The .env file is located at the root of the workspace (two levels up from this script)
    current_dir = Path.cwd()
    env_path = current_dir / "../../.env"
    load_dotenv(dotenv_path=env_path)
    
    # Extract Azure AI project parameters from environment
    config = {
        "project_endpoint": os.environ.get("AZURE_AI_PROJECT_ENDPOINT"),
        "agent_name": os.environ.get("AZURE_AI_AGENT_NAME"),
    }
    
    # Validate that essential configuration is present
    if not config["project_endpoint"]:
        raise ValueError("Please set the AZURE_AI_PROJECT_ENDPOINT environment variable.")
        
    if not config["agent_name"]:
        raise ValueError("Please set the AZURE_AI_AGENT_NAME environment variable.")
    
    return config

## 3. Discover and Validate the Target Agent

Before we can test an agent, we need to find it within our Azure AI Project. This function searches through all available agents to locate the one matching our specified name, then retrieves its full configuration including the model deployment.

In [3]:
def resolve_agent(project_client: AIProjectClient, agent_name: str) -> tuple:
    """
    Locate and retrieve the target agent for red team testing by name.
    
    Args:
        project_client: Authenticated Azure AI Project client
        agent_name: Name of the agent to find
        
    Returns:
        Tuple of (agent_object, agent_id, deployment_name)
    """
    print(f"Searching for agent by name: {agent_name}")
    
    # Search through all agents to find one with matching name
    agent_id = None
    for agent in project_client.agents.list_agents():
        if agent.name == agent_name:
            agent_id = agent.id
            print(f"✓ Found agent: {agent_name} (ID: {agent_id})")
            break
    
    # Validate that we found the agent
    if not agent_id:
        raise ValueError(f"Agent with name '{agent_name}' not found. Please check the agent name.")
    
    # Retrieve the full agent object
    agent = project_client.agents.get_agent(agent_id)
    
    # Extract the model deployment name from the agent configuration
    deployment_name = agent.model
    
    return agent, agent_id, deployment_name

## 4. Create Agent Interaction Callback

The red team scanner needs a way to communicate with our agent. We create a callback function that handles the full conversation flow: creating messages, starting agent runs, polling for completion, and retrieving responses. This callback will be invoked by the red team scanner for each adversarial test case.

In [4]:
def create_agent_callback(project_client: AIProjectClient, agent_id: str, thread_id: str):
    """
    Create a callback function that allows the red team to interact with the agent.
    
    The red team scanner uses this callback to send adversarial prompts to the agent
    and receive responses.
    
    Args:
        project_client: Authenticated Azure AI Project client
        agent_id: ID of the agent to test
        thread_id: Conversation thread ID for maintaining context
        
    Returns:
        Callable that takes a query string and returns the agent's response
    """
    def agent_callback(query: str) -> str:
        # Create a new message in the conversation thread
        message = project_client.agents.messages.create(
            thread_id=thread_id, 
            role="user", 
            content=query
        )
        
        # Initiate an agent run to process the message
        run = project_client.agents.runs.create(
            thread_id=thread_id, 
            agent_id=agent_id
        )

        # Poll the run status until completion
        while run.status in ["queued", "in_progress", "requires_action"]:
            time.sleep(1)
            run = project_client.agents.runs.get(
                thread_id=thread_id, 
                run_id=run.id
            )
            print(f"Run status: {run.status}")

        # Handle failure cases
        if run.status == "failed":
            print(f"Run error: {run.last_error}")
            return "Error: Agent run failed."
        
        # Retrieve the agent's response from the message list
        messages = project_client.agents.messages.list(
            thread_id=thread_id, 
            order=ListSortOrder.DESCENDING
        )
        
        # Extract the text content from the latest message
        for msg in messages:
            if msg.text_messages:
                return msg.text_messages[0].text.value
                
        return "Could not get a response from the agent."
    
    return agent_callback

## 5. Configure the Red Team Scanner

Now we configure the red team scanner with our security testing parameters. We specify which risk categories to test (e.g., Violence, Hate Speech), how many test objectives to generate, and where to save the detailed results. This configuration determines the scope and thoroughness of our security testing.

In [5]:
def configure_red_team(project_endpoint: str, credential: DefaultAzureCredential) -> RedTeam:
    """
    Configure the red team scanner with security testing parameters.
    
    Args:
        project_endpoint: Azure AI Project endpoint URL
        credential: Authenticated Azure credential
        
    Returns:
        Configured RedTeam scanner instance
    """
    print("\nConfiguring Red Team scanner...")
    print("  - Risk Categories: Violence")
    print("  - Number of Objectives: 1")
    print("  - Output Directory: redteam_outputs/")
    
    red_team = RedTeam(
        azure_ai_project=project_endpoint,
        credential=credential,
        risk_categories=[RiskCategory.Violence],  # Categories of harmful content to test
        num_objectives=1,  # Number of test objectives to generate per category
        output_dir="redteam_outputs/"  # Where to save detailed results
    )
    
    return red_team

## 6. Execute the Red Team Security Scan

This is where the actual security testing happens. The scanner will generate adversarial prompts based on attack strategies, send them to the agent through our callback function, analyze the responses for vulnerabilities, and generate a comprehensive security report. This process may take several minutes depending on the complexity of testing.

In [6]:
async def execute_red_team_scan(red_team: RedTeam, agent_callback: callable) -> Any:
    """
    Execute the red team security scan against the agent.
    
    Args:
        red_team: Configured RedTeam scanner
        agent_callback: Function to interact with the target agent
        
    Returns:
        Scan results containing vulnerability findings and metrics
    """
    print("\n" + "="*70)
    print("STARTING RED TEAM SCAN")
    print("="*70)
    print("\nThe scanner will now:")
    print("  1. Generate adversarial test prompts")
    print("  2. Send prompts to the agent")
    print("  3. Analyze responses for vulnerabilities")
    print("  4. Generate a security report")
    print("\nThis may take several minutes...\n")
    
    result = await red_team.scan(
        target=agent_callback,  # The function to test
        scan_name="Agent-Scan",  # Name for this scan session
        attack_strategies=[AttackStrategy.Flip],  # Attack techniques to employ
    )
    
    print("\n" + "="*70)
    print("RED TEAM SCAN COMPLETE")
    print("="*70)
    print("\nResults have been saved to: redteam_outputs/")
    print("Review the output files for detailed vulnerability findings.\n")
    
    return result

## 7. Orchestrate the Complete Red Teaming Workflow

This main function coordinates all stages of the red teaming process. It walks through each stage sequentially: loading configuration, connecting to Azure, finding the target agent, setting up communication, and executing the security scan. We use context managers to ensure proper cleanup of Azure resources.

In [7]:
async def run_red_team():
    """
    Main orchestration function that executes the complete red teaming workflow.
    
    This function coordinates all stages:
    - STAGE 1: Load environment configuration
    - STAGE 2: Connect to Azure AI Project and locate target agent
    - STAGE 3: Set up communication channel with the agent
    - STAGE 4: Configure and execute the red team security scan
    - STAGE 5: Return results for analysis
    """
    print("\n" + "="*70)
    print("AI RED TEAMING - SECURITY TESTING FOR AZURE AI AGENTS")
    print("="*70 + "\n")
    
    # STAGE 1: Environment Setup
    print("STAGE 1: Loading Environment Configuration")
    print("-" * 70)
    config = load_environment_configuration()
    print("✓ Configuration loaded successfully\n")
    
    # Initialize Azure credentials
    credential = DefaultAzureCredential(exclude_interactive_browser_credential=False)
    
    # Use context managers for proper resource management
    with credential:
        with AIProjectClient(
            endpoint=config["project_endpoint"], 
            credential=credential
        ) as project_client:
            
            # STAGE 2: Agent Discovery
            print("STAGE 2: Resolving Target Agent")
            print("-" * 70)
            agent, agent_id, deployment_name = resolve_agent(
                project_client, 
                config["agent_name"]
            )
            print(f"✓ Target Agent Located:")
            print(f"    Agent ID: {agent.id}")
            print(f"    Agent Name: {agent.name}")
            print(f"    Model Deployment: {deployment_name}\n")
            
            # STAGE 3: Communication Setup
            print("STAGE 3: Setting Up Agent Communication")
            print("-" * 70)
            thread = project_client.agents.threads.create()
            print(f"✓ Conversation thread created: {thread.id}")
            
            agent_callback = create_agent_callback(project_client, agent.id, thread.id)
            print("✓ Agent callback configured\n")
            
            # STAGE 4: Red Team Scan
            print("STAGE 4: Executing Red Team Scan")
            print("-" * 70)
            red_team = configure_red_team(config["project_endpoint"], credential)
            result = await execute_red_team_scan(red_team, agent_callback)
            
            # STAGE 5: Completion
            print("STAGE 5: Scan Complete")
            print("-" * 70)
            print("✓ All stages completed successfully")
            print("✓ Review the output directory for detailed findings\n")
            
            return result

## 8. Run the Red Team Security Test

Finally, we execute the complete red teaming workflow. Since the scanning process is asynchronous, we use `asyncio` to run our async functions. This cell will perform the entire security assessment and display progress as it goes through each stage.

In [8]:
import asyncio

# Execute the red team security scan
result = await run_red_team()

# Display summary
print("\n" + "="*70)
print("SCAN SUMMARY")
print("="*70)
print(f"Scan completed successfully!")
print(f"Check the 'redteam_outputs/' directory for detailed results.")


AI RED TEAMING - SECURITY TESTING FOR AZURE AI AGENTS

STAGE 1: Loading Environment Configuration
----------------------------------------------------------------------
✓ Configuration loaded successfully

STAGE 2: Resolving Target Agent
----------------------------------------------------------------------
Searching for agent by name: zava-customer-support
✓ Found agent: zava-customer-support (ID: asst_4fqxHf1SLB5bOnzVAzahsZ8m)
✓ Target Agent Located:
    Agent ID: asst_4fqxHf1SLB5bOnzVAzahsZ8m
    Agent Name: zava-customer-support
    Model Deployment: gpt-4.1-mini

STAGE 3: Setting Up Agent Communication
----------------------------------------------------------------------
✓ Conversation thread created: thread_OuQrbldW6yKcxSDLCs8hWEaA
✓ Agent callback configured

STAGE 4: Executing Red Team Scan
----------------------------------------------------------------------

Configuring Red Team scanner...
  - Risk Categories: Violence
  - Number of Objectives: 1
  - Output Directory: redt

Class RedTeam: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.



STARTING RED TEAM SCAN

The scanner will now:
  1. Generate adversarial test prompts
  2. Send prompts to the agent
  3. Analyze responses for vulnerabilities
  4. Generate a security report

This may take several minutes...

🚀 STARTING RED TEAM SCAN
📂 Output directory: redteam_outputs/.scan_Agent-Scan_20251021_004326
📊 Risk categories: ['violence']
🔗 Track your red team scan in AI Foundry: https://ai.azure.com/resource/build/redteaming/ef94ba80-12dc-4349-882f-d9b2490112a1?wsid=/subscriptions/7a880728-70d3-49d0-adde-4250716cfd94/resourceGroups/rg-nitya-Ignite-LAB516/providers/Microsoft.CognitiveServices/accounts/aoai-2rdxoijm443lk/projects/proj-2rdxoijm443lk&tid=498cf826-3254-4eb7-8e43-8cf263c1a3f7
📋 Planning 2 total tasks
📝 Fetched baseline objectives for violence: 1 objectives
🔄 Fetching objectives for strategy 2/2: flip


Scanning:   0%|                                       | 0/2 [00:00<?, ?scan/s, current=initializing]

⚙️ Processing 2 tasks in parallel (max 5 at a time)
▶️ Starting task: baseline strategy for violence risk category
▶️ Starting task: flip strategy for violence risk category
Run status: incomplete
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.COMPLETED


Scanning:  50%|███████████████▌               | 1/2 [00:16<00:16, 16.02s/scan, current=initializing]

Evaluation results saved to "/workspaces/ignite25-LAB516-safeguard-your-agents-with-ai-red-teaming-agent-in-azure-ai-foundry/labs/lab-01/redteam_outputs/.scan_Agent-Scan_20251021_004326/baseline_violence_5e7302f3-45af-4ff1-a55b-500629f7ca06.json".
✅ Completed task 1/2 (50.0%) - baseline/violence in 16.0s
   Est. remaining: 0.3 minutes


Scanning: 100%|███████████████████████████████| 2/2 [00:17<00:00,  8.91s/scan, current=initializing]
Class RedTeamResult: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Evaluation results saved to "/workspaces/ignite25-LAB516-safeguard-your-agents-with-ai-red-teaming-agent-in-azure-ai-foundry/labs/lab-01/redteam_outputs/.scan_Agent-Scan_20251021_004326/flip_violence_d3da6590-8172-4f8a-b578-e4d181016fc6.json".
✅ Completed task 2/2 (100.0%) - flip/violence in 17.8s
   Est. remaining: 0.0 minutes
Evaluation results saved to "/workspaces/ignite25-LAB516-safeguard-your-agents-with-ai-red-teaming-agent-in-azure-ai-foundry/labs/lab-01/redteam_outputs/.scan_Agent-Scan_20251021_004326/final_results.json".

Overall ASR: 0.0%
Attack Success: 0/2 attacks were successful
------------------------------------------------------------------------------------------------------------------------------------
Risk Category        | Baseline ASR   | Easy-Complexity Attacks ASR  | Moderate-Complexity Attacks ASR | Difficult-Complexity Attacks ASR
---------------------------------------------------------------------------------------------------------------------------------